In [29]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy
import numpy as np
import seaborn as sns

Objetivos do Script:

Esse script tem como objetivo ler os dados que foram capturados da nossa amostra piloto e realizar alguns estudos em cima desses dados. 

Os 6 primeiros tópicos que eu separei abaixo são utilizados na grande maioria dos scripts de tratamento de dados desse estudo. 
Deixei padronizado para que seja possivel tratar simulações com um dispositivo ou vários.

Sumário:
  - Leitura dos dados
  - Separar publicações e confirmação de entrega por dispositivo simulado.
  - Criação de um dataframe com informações capturadas. 
  - Medidas de tendência centrar por dispositivo
  - Medidas de tendência centrar de todos os dispositivos.
  - Gráfico de dispersão da amostra.
  - Calculo do tamanho da amostra.

Como estamos trabalhando com uma amostra piloto temos como objetivo calcular o tamanho das amostras isso será realizado na ultima parte desse script.


In [30]:
data = pd.read_csv("./data.csv")
ports = data["Source Port"].unique()
ports = ports[ports !=1883]
data

,No.,Time,Source,Destination,Protocol,Length,Source Port,Message Identifier,Destination Port,Info
0,213,19.166036,192.168.15.8,130.185.238.170,MQTT,68,55897,NaN,1883,Ping Request
1,214,19.252044,130.185.238.170,192.168.15.8,MQTT,68,1883,NaN,55897,Ping Response
2,243,20.865097,192.168.15.8,130.185.238.170,MQTT,68,45035,NaN,1883,Ping Request
3,245,20.945240,130.185.238.170,192.168.15.8,MQTT,68,1883,NaN,45035,Ping Response
4,247,21.004982,192.168.15.8,130.185.238.170,MQTT,68,58693,NaN,1883,Ping Request
...,...,...,...,...,...,...,...,...,...,...
12525,39244,806.981741,130.185.238.170,192.168.15.8,MQTT,337,1883,NaN,36243,Publish Message [devices-20]
12526,39246,807.060753,130.185.238.170,192.168.15.8,MQTT,70,1883,200,36243,Publish Ack (id=200)
12527,39279,808.751297,192.168.15.8,130.185.238.170,MQTT,339,36243,201,1883,Publish Message (id=201) [devices-20]
12528,39280,808.832865,130.185.238.170,192.168.15.8,MQTT,337,1883,NaN,36243,Publish Message [devices-20]


In [31]:
#  - Separar publicações e confirmação de entrega por dispositivo simulado.
devices = {}
#  : devices será um dicionario que irá ter uma chave referente ao device e o valor será um dataframe referente aos respectivos devices 
for port in ports:  
  data = data.loc[ ~(data["Message Identifier"] == "1,2")]
  # data["Message Identifier"].dropna(replace=True)
  array_publish = data.loc[data["Source Port"] == port].dropna(subset=['Message Identifier'])
  array_publish_ack = data.loc[data["Destination Port"] == port].dropna(subset=['Message Identifier'])
  device = {"publish":array_publish, "publish_ack" : array_publish_ack}
  devices[port] = device

In [33]:
ports = ports[4:]



In [34]:
# - Criação de um dataframe com informações capturadas acima. 
data = {}
for port in ports:
    RTT = pd.DataFrame(columns=["Time_publish", "Time_ack"])
    for index, row in devices[port]["publish"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_publish"] = time_publish 

    for index, row in devices[port]["publish_ack"].iterrows():
        id = int(row["Message Identifier"])
        time_publish = row["Time"]
        RTT.loc[id, "Time_ack"] = time_publish
    RTT.loc[id, "Time_ack"] = time_publish
    RTT["RTT"]  =(RTT["Time_ack"] - RTT["Time_publish"] )* 1000
    data[port] = RTT


## Medidas de Tendência Central Por Device

In [35]:
i=1
data_mean = []
data_median = []
data_std = []
data_min = []
data_max = []

for port in ports:
  data_mean.append(data[port]["RTT"].mean())
  data_median.append(data[port]["RTT"].mean())
  data_std.append(data[port]["RTT"].mean())
  data_min.append(data[port]["RTT"].mean())
  data_max.append(data[port]["RTT"].mean())

  print("(Device-{})Mean Round Trip Time = {}".format(i ,data[port]["RTT"].mean()))
  print("(Device-{})Median Round Trip Time = {}".format(i ,data[port]["RTT"].median()))
  print("(Device-{})Standard deviation  Round Trip Time = {}".format(i ,data[port]["RTT"].std()))
  print("(Device-{})Min Round Trip Time = {}".format(i ,data[port]["RTT"].min()))
  print("(Device-{})Max Round Trip Time = {}".format(i ,data[port]["RTT"].max()))
  print("---------------------------------------------------------------")

  i+=1

(Device-1)Mean Round Trip Time = 160.62158335678188
(Device-1)Median Round Trip Time = 160.5526889999993
(Device-1)Standard deviation  Round Trip Time = 1.3904669951221205
(Device-1)Min Round Trip Time = 158.09481899998445
(Device-1)Max Round Trip Time = 166.39867099999606
---------------------------------------------------------------
(Device-2)Mean Round Trip Time = 153.3549716532735
(Device-2)Median Round Trip Time = 152.5987629999861
(Device-2)Standard deviation  Round Trip Time = 6.134321095027122
(Device-2)Min Round Trip Time = 150.27907500001447
(Device-2)Max Round Trip Time = 228.669463000017
---------------------------------------------------------------
(Device-3)Mean Round Trip Time = 154.12984488944892
(Device-3)Median Round Trip Time = 153.92628500001138
(Device-3)Standard deviation  Round Trip Time = 2.410270883053983
(Device-3)Min Round Trip Time = 151.51224800001728
(Device-3)Max Round Trip Time = 178.09660500000746
------------------------------------------------------

# CÁLCULO DO TAMANHO DA AMOSTRA

In [36]:
nivel_de_significancia = 0.05

In [37]:
nivel_de_confiança = 1 - nivel_de_significancia
nivel_de_confiança

0.95

In [38]:
#Calculando o z com scipy
z = scipy.stats.norm.ppf(0.975)
z

1.959963984540054

Com nosso z calculado podemos seguir adiante

### $$z = 1.959963984540054 $$

Abaixo segue algumas formulas que iremos utilizar para calcular o numero de amostra



## $$n = \left(z\frac{s}{e}\right)^2$$


$$e = \left(e *  \right)^2$$



Onde:

$z$ = variável normal padronizada

$s$ = desvio padrão amostral

$e$ = erro inferencial


### Calculando a desvio padrão $s$

In [39]:
desvio_padrao_amostral = RTT["RTT"].std()
desvio_padrao_amostral

6.861313988662176

### Calculando o Erro

In [40]:
media = RTT["RTT"][:-2].mean()
e = 0.05 * media
e

8.092436197236266

### Calculando o tamanho da nossa amostra

In [41]:
n = (z * (desvio_padrao_amostral/e))**2
n

2.7615454932661554

In [42]:
n = (z * (desvio_padrao_amostral/e))**2

In [43]:
n

2.7615454932661554

In [44]:
RTT

,Time_publish,Time_ack,RTT
3,70.643735,70.804083,160.347901
4,71.929164,72.088366,159.202127
5,77.670833,77.832287,161.454324
6,81.607962,81.768448,160.485441
7,87.377795,87.540471,162.676809
...,...,...,...
199,804.888811,805.048706,159.894969
200,806.901585,807.060753,159.167981
201,808.751297,808.913772,162.475219
1,NaN,65.093033,NaN


In [ ]:
data[37665][:-2]

,Time_publish,Time_ack,RTT
3,2307.359166,2307.518452,159.286203
4,2312.077159,2312.238383,161.224181
5,2317.926067,2318.08651,160.443781
6,2323.049268,2323.21055,161.281924
7,2326.961367,2327.122621,161.253469
...,...,...,...
197,3040.939875,3041.100178,160.302985
198,3042.448657,3042.60658,157.922825
199,3047.067165,3047.226274,159.109616
200,3052.704503,3052.862299,157.796541
